In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()
df.columns

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

In [ ]:
vine_filtered_df = vine_df.filter((vine_df["total_votes"] > 20) & (vine_df["helpful_votes"]/vine_df["total_votes"] > 0.5))
vine_filtered_df.show()

In [ ]:
vine_paid_df = vine_filtered_df.filter(vine_filtered_df["vine"] == "Y")
vine_paid_df.show()

In [ ]:
vine_unpaid_df = vine_filtered_df.filter(vine_filtered_df["vine"] == "N")
vine_unpaid_df.show()

In [ ]:
vine_paid_df.count()

In [ ]:
vine_unpaid_df.count()

In [ ]:
vine_paid_df.filter(vine_paid_df["star_rating"] == 5).count()

In [ ]:
vine_unpaid_df.filter(vine_paid_df["star_rating"] == 5).count()

In [ ]:
(vine_paid_df.filter(vine_paid_df["star_rating"] == 5).count())/(vine_paid_df.count())*100

In [ ]:
(vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == 5).count())/(vine_unpaid_df.count())*100